**RNN on Custom dataset**

In [ ]:
import pandas as pd
df=pd.read_csv('yelp.csv')
df.head(4)

,Unnamed: 0,sentence,label,source
0,0,Wow... Loved this place.,1,yelp
1,1,Crust is not good.,0,yelp
2,2,Not tasty and the texture was just nasty.,0,yelp
3,3,Stopped by during the late May bank holiday of...,1,yelp


splitting in test train

In [ ]:
x=df['sentence']
y=df['label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
print("shape of X_train: ",X_train.shape)
print("shape of X_test: ",X_test.shape)

print("shape of y_train: ",y_train.shape)
print("shape of y_test: ",y_test.shape)

shape of X_train:  (800,)
shape of X_test:  (200,)
shape of y_train:  (800,)
shape of y_test:  (200,)


In [ ]:
print(X_train[6]) 
print(y_train[6])


Honeslty it didn't taste THAT fresh.)
0


Custom word embedding (conversion word to vector using keras : texts_to_sequences)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000,lower=True)
tokenizer.fit_on_texts(X_train)



X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


independent and dependent variable after word to vector

In [ ]:
print(X_train[6])
print(y_train[6])

[3, 75, 21, 25]
0


Padding 

In [ ]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 100  #We set the maximum size of each list to 100.


X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[3, :]) 

[165   9  51 351  20  10 241   2 776 105   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


One-Hot encoding

In [ ]:
from keras.utils import to_categorical
num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_train[0])

(800, 2)
[1. 0.]


changing dimentionality of input to 3-d as needed for Rnn

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SimpleRNN
from keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


#(samples, timesteps, features)
#(batch_size, timesteps, input_dim)
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)  #(750, 100, 1)

X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

(800, 100, 1)
(200, 100, 1)


model function 

In [ ]:
num_classes=2

def vanilla_rnn():
    model = Sequential()
    #model.add(SimpleRNN(50, input_shape = (X_train.shape[1], X_train.shape[2],), return_sequences = False))
    model.add(SimpleRNN(50, input_shape = (maxlen,1), return_sequences = False))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model training

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
#hyperparameters
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 100, batch_size = 50)
model.fit(X_train, y_train)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 50)                2600      
                                                                 
 dense_2 (Dense)             (None, 2)                 102       
                                                                 
 activation_2 (Activation)   (None, 2)                 0         
                                                                 
Total params: 2,702
Trainable params: 2,702
Non-trainable params: 0
_________________________________________________________________


<ipython-input-102-896c73fb37fa>:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = vanilla_rnn, epochs = 100, batch_size = 50)
/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
16/16 [==============================] - 1s 18ms/step - loss: 0.7010 - accuracy: 0.4863
Epoch 2/100
16/16 [==============================] - 0s 17ms/step - loss: 0.6924 - accuracy: 0.5300
Epoch 3/100
16/16 [==============================] - 0s 17ms/step - loss: 0.6920 - accuracy: 0.5325
Epoch 4/100
16/16 [==============================] - 0s 18ms/step - loss: 0.6944 - accuracy: 0.4888
Epoch 5/100
16/16 [==============================] - 0s 28ms/step - loss: 0.6909 - accuracy: 0.5225
Epoch 6/100
16/16 [==============================] - 1s 33ms/step - loss: 0.6885 - accuracy: 0.5612
Epoch 7/100
16/16 [==============================] - 1s 32ms/step - loss: 0.6977 - accuracy: 0.5238
Epoch 8/100
16/16 [==============================] - 1s 33ms/step - loss: 0.6919 - accuracy: 0.5213
Epoch 9/100
16/16 [==============================] - 1s 34ms/step - loss: 0.6967 - accuracy: 0.5100
Epoch 10/100
16/16 [==============================] - 0s 30ms/step - loss: 0.6913 - accuracy: 0.5412

metrics- accuracy

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

print(accuracy_score(y_pred, y_test_))

7/7 [==============================] - 0s 10ms/step
0.525


Testing model on New input

In [ ]:

a= ['Waitress was horrible, bad in service.']
#a = [" i love their service"]
#applying tokenizer and padding to new input
a = tokenizer.texts_to_sequences(a)
a=np.array(a)
a = pad_sequences(a, padding='post', maxlen=maxlen)

a = a.reshape((a.shape[0], a.shape[1], 1))
print(a.shape)

prediction = model.predict(np.array(a))
print(prediction) # 0 means negative and 1 means positive

(1, 100, 1)
1/1 [==============================] - 0s 36ms/step
[0]
